In [46]:
import os
import pandas as pd
import numpy as np
import missingno as msno
import plotly.express as px
from datetime import datetime
import math


In [47]:
#PASTA BASE COM OS ARQUIVOS CSV
pasta_arquivos_csv = "./0_bases_originais_CSV/"

In [48]:
# CRIANDO O DATAFRAME COM LISTA DOS MESES PARA FAZER O LOOPING PRA FICAR MAIS FÁCIL DE LER E CONCATENAR
dfs = [pd.read_csv(os.path.join(pasta_arquivos_csv, arquivo), sep=';', encoding='utf-8')
for arquivo in os.listdir(pasta_arquivos_csv)
if arquivo.endswith(".csv")]


In [49]:

# CONCATENANDO TUDO E CRIANDO UM ÚNICO ARQUIVO CSV
dados_geral = pd.concat(dfs, ignore_index=True)


In [50]:
# EXIBE TODAS AS COLUNAS INDEPENDENTE DA QUANTIDADE
pd.set_option('display.max_columns', None)

In [51]:
# DEIXA O NOME DAS COLUNAS EM MINUSCULO
dados_geral.columns = dados_geral.columns.str.lower()

In [52]:
dados_geral['modelo_veic'] = dados_geral['descr_marca_veiculo'].str.split('/').str[1]
dados_geral['marca_veic'] = dados_geral['descr_marca_veiculo'].str.split('/').str[0]

In [53]:
dados_geral['dataocorrencia'] = dados_geral['dataocorrencia'].astype(str)
dados_geral['mes'] = dados_geral['dataocorrencia'].str[3:4]
dados_geral['ano'] = dados_geral['dataocorrencia'].str[4:10]
dados_geral['mes_ocorrencia'] = dados_geral['mes'] + dados_geral['ano']

In [54]:
dados_geral['ano_modelo']=dados_geral.ano_modelo.astype(str)
dados_geral['ano_modelo']=dados_geral.ano_modelo.str[:4]

In [55]:
dados_geral = dados_geral.drop(columns=['ano_bo','num_bo','numero_boletim','bo_iniciado','bo_emitido','dataocorrencia','horaocorrencia','datacomunicacao',
'dataelaboracao','bo_autoria','flagrante','numero_boletim_principal','logradouro','numero','bairro','uf','latitude','longitude','exame','solucao',
'delegacia_nome','delegacia_circunscricao','especie','desdobramento','status','tipopessoa','vitimafatal','sexo','datanascimento','idade',
'estadocivil','profissao','grauinstrucao','naturezavinculada','tipovinculo','relacionamento', 'parentesco','placa_veiculo','uf_veiculo',
'descr_marca_veiculo','ano_fabricacao','naturalidade','nacionalidade','corcutis', 'quant_celular','marca_celular','mes','ano'])

In [56]:
dados_geral

,peridoocorrencia,cidade,descricaolocal,rubrica,cidade_veiculo,descr_cor_veiculo,ano_modelo,descr_tipo_veiculo,modelo_veic,marca_veic,mes_ocorrencia
0,A NOITE,S.ANDRE,Via Pública,Furto (art. 155) - VEICULO,SÃO BERNARDO DO CAMP,Prata,nan,AUTOMOVEL,NISSAN MARCH 10S FLEX,I,03/2022
1,A NOITE,S.ANDRE,Via pública,Furto (art. 155) - VEICULO,S.BERNARDO DO CAMPO,Prata,2022,AUTOMOVEL,HB20 10M VISION,HYUNDAI,03/2022
2,A NOITE,GUARULHOS,Via Pública,Furto (art. 155) - VEICULO,GUARULHOS,Azul,nan,MOTOCICLO,FACTOR YBR125 E,YAMAHA,03/2022
3,DE MADRUGADA,CARAPICUIBA,Via pública,Localização/Apreensão e Entrega de veículo,NaN,NaN,0.0,NaN,NaN,NaN,04/2022
4,DE MADRUGADA,CARAPICUIBA,Via pública,Localização/Apreensão e Entrega de veículo,CARAPICUIBA,Branco,2015,MOTONETA,BIZ 125 EX,HONDA,04/2022
...,...,...,...,...,...,...,...,...,...,...,...
35036,A TARDE,S.ANDRE,Via pública,Furto (art. 155) - VEICULO,S.ANDRE,Prata,nan,AUTOMOVEL,HYUNDAI I30 2.0,I,09/2022
35037,A TARDE,GUARULHOS,Via pública,Furto (art. 155) - VEICULO,GUARULHOS,Preta,nan,AUTOMOVEL,KIA PICANTO EX3 1.0L,I,09/2022
35038,A NOITE,S.ANDRE,Estacionamento público,Furto (art. 155) - VEICULO,S.PAULO,Branco,nan,MOTOCICLO,CG 160 FAN,HONDA,09/2022
35039,DE MADRUGADA,GUARULHOS,Via pública,Furto (art. 155) - VEICULO,POA,Azul,nan,AUTOMOVEL,KA GL,FORD,09/2022


In [57]:
'''# SALVANDO OS ARQUIVOS COMPILADOS E CRIANDO UM NOVO ARQUIVO DENTRO DA PASTA BASE TRATADAS
salvar_pasta = '../1_bases_tratadas/dados_GERAL.csv'
dados_geral.to_csv(salvar_pasta, index=False)'''

"# SALVANDO OS ARQUIVOS COMPILADOS E CRIANDO UM NOVO ARQUIVO DENTRO DA PASTA BASE TRATADAS\nsalvar_pasta = '../1_bases_tratadas/dados_GERAL.csv'\ndados_geral.to_csv(salvar_pasta, index=False)"

In [58]:
'''
A PARTE DE DICAS DE TRATAMENTO COMEÇA DAQUI PARA BAIXO!
ONDE VAMOS CRIAR:

- MONTAR UM GRÁFICO DE DISPERSÃO NO LUGAR DO DE OUTLIER
trocar por px.scatter()

- FAZENDO UM TIPO DE "LEFT JOIN" PARA INCLUIR A COLUNA AUXILIAR CRIADA NA BASE TRATADA

- CRIANDO UM BANCO DE DADOS AUXILIAR, PARA TER VALORES NUMERICOS PARA OS GRÁFICOS

- ESCOLHER OUTRAS COLUNAS QUE FAÇAM SENTIDO E CRIAR OUTRO BANCO AUXILIAR E INCLUINDO ELE A BASE TRATADA


'''

'\nA PARTE DE DICAS DE TRATAMENTO COMEÇA DAQUI PARA BAIXO!\nONDE VAMOS CRIAR:\n\n- MONTAR UM GRÁFICO DE DISPERSÃO NO LUGAR DO DE OUTLIER\ntrocar por px.scatter()\n\n- FAZENDO UM TIPO DE "LEFT JOIN" PARA INCLUIR A COLUNA AUXILIAR CRIADA NA BASE TRATADA\n\n- CRIANDO UM BANCO DE DADOS AUXILIAR, PARA TER VALORES NUMERICOS PARA OS GRÁFICOS\n\n- ESCOLHER OUTRAS COLUNAS QUE FAÇAM SENTIDO E CRIAR OUTRO BANCO AUXILIAR E INCLUINDO ELE A BASE TRATADA\n\n\n'

In [59]:
# criar bancos auxiliares com agrupamentos
banco_auxiliar1 = dados_geral.groupby(['descricaolocal'])['descricaolocal'].agg(['count'])

In [60]:
# renomendo a coluna com os novos valores
banco_auxiliar1 = banco_auxiliar1.reset_index().rename(columns={'count':'contagem_descricaolocal'})

In [61]:
banco_auxiliar1

,descricaolocal,contagem_descricaolocal
0,Area não ocupada,43
1,Centro Comerc./Empresarial,252
2,Comércio e serviços,603
3,Condominio Comercial,36
4,Condominio Residencial,330
5,Entidade assistencial,1
6,Escritório,35
7,Estabelecimento bancário,1
8,Estabelecimento de ensino,32
9,Estabelecimento industrial,2


In [62]:
#FAZENDO UM "LEFT JOIN" PARA INCLUIR A COLUNA AUXILIAR CRIADA NA BASE TRATADA
dados_geral = pd.merge(dados_geral, banco_auxiliar1, how='left', on='descricaolocal')

In [63]:
# criar bancos auxiliares com agrupamentos
banco_auxiliar2 = dados_geral.groupby(['modelo_veic'])['modelo_veic'].agg(['count'])

In [64]:
# renomendo a coluna com os novos valores
banco_auxiliar2 = banco_auxiliar2.reset_index().rename(columns={'count':'contagem_modelo_veic'})

In [65]:
banco_auxiliar2

,modelo_veic,contagem_modelo_veic
0,SAVEIRO MAIA AMB,2
1,10.160 DRC 4X2,1
2,11.180 DRC 4X2,4
3,12.140 H,1
4,13.180 CNM,1
...,...,...
2512,ZAFIRA ELEGANCE,3
2513,ZAFIRA ELITE,6
2514,ZAFIRA EXPRESSION,7
2515,ZIG PLUS,1


In [66]:
#FAZENDO UM "LEFT JOIN" PARA INCLUIR A COLUNA AUXILIAR CRIADA NA BASE TRATADA
dados_geral = pd.merge(dados_geral, banco_auxiliar2, how='left', on='modelo_veic')

In [67]:
dados_geral = dados_geral.rename(columns={
    'peridoocorrencia': 'Período Ocorrência',
    'cidade': 'Cidade',
    'descricaolocal': 'Descrição Local',
    'rubrica': 'Rubrica',
    'cidade_veiculo': 'Cidade Veículo',
    'descr_cor_veiculo': 'Cor Veículo',
    'ano_modelo': 'Ano Modelo',
    'descr_tipo_veiculo': 'Tipo Veículo',
    'modelo_veic': 'Modelo Veículo',
    'marca_veic': 'Marca Veículo',
    'mes_ocorrencia': 'Mês Ocorrência',
    'contagem_desc_local': 'Contagem Descrição Local',
    'contagem_modelo_veiculo': 'Contagem Modelo Veículo'
})

In [68]:
dados_geral

,Período Ocorrência,Cidade,Descrição Local,Rubrica,Cidade Veículo,Cor Veículo,Ano Modelo,Tipo Veículo,Modelo Veículo,Marca Veículo,Mês Ocorrência,contagem_descricaolocal,contagem_modelo_veic
0,A NOITE,S.ANDRE,Via Pública,Furto (art. 155) - VEICULO,SÃO BERNARDO DO CAMP,Prata,nan,AUTOMOVEL,NISSAN MARCH 10S FLEX,I,03/2022,6309,9.0
1,A NOITE,S.ANDRE,Via pública,Furto (art. 155) - VEICULO,S.BERNARDO DO CAMPO,Prata,2022,AUTOMOVEL,HB20 10M VISION,HYUNDAI,03/2022,22353,27.0
2,A NOITE,GUARULHOS,Via Pública,Furto (art. 155) - VEICULO,GUARULHOS,Azul,nan,MOTOCICLO,FACTOR YBR125 E,YAMAHA,03/2022,6309,6.0
3,DE MADRUGADA,CARAPICUIBA,Via pública,Localização/Apreensão e Entrega de veículo,NaN,NaN,0.0,NaN,NaN,NaN,04/2022,22353,NaN
4,DE MADRUGADA,CARAPICUIBA,Via pública,Localização/Apreensão e Entrega de veículo,CARAPICUIBA,Branco,2015,MOTONETA,BIZ 125 EX,HONDA,04/2022,22353,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35036,A TARDE,S.ANDRE,Via pública,Furto (art. 155) - VEICULO,S.ANDRE,Prata,nan,AUTOMOVEL,HYUNDAI I30 2.0,I,09/2022,22353,216.0
35037,A TARDE,GUARULHOS,Via pública,Furto (art. 155) - VEICULO,GUARULHOS,Preta,nan,AUTOMOVEL,KIA PICANTO EX3 1.0L,I,09/2022,22353,3.0
35038,A NOITE,S.ANDRE,Estacionamento público,Furto (art. 155) - VEICULO,S.PAULO,Branco,nan,MOTOCICLO,CG 160 FAN,HONDA,09/2022,962,1278.0
35039,DE MADRUGADA,GUARULHOS,Via pública,Furto (art. 155) - VEICULO,POA,Azul,nan,AUTOMOVEL,KA GL,FORD,09/2022,22353,17.0


In [69]:
# SALVANDO OS ARQUIVOS COMPILADOS E CRIANDO UM NOVO ARQUIVO DENTRO DA PASTA BASE TRATADAS
salvar_pasta = './1_bases_tratadas/dados_GERAL.csv'
dados_geral.to_csv(salvar_pasta, index=False)